# Ham ya da Spam?

🎯 Bu görevin amacı, e-postaları **spam (1)** veya **normal e-posta (0)** olarak sınıflandırmaktır.

🧹 İlk olarak, bu metin verilerine **temizleme (cleaning)** teknikleri uygulanacaktır.

👩🏻‍🔬 Ardından, temizlenmiş metinler **sayısal bir gösterime** dönüştürülecektir.

✉️ Son olarak, her bir e-postayı spam mı yoksa normal mi olduğunu sınıflandırmak için  
***Multinomial Naive Bayes*** modeli uygulanacaktır.


## (0) NTLK kütüphanesi (Doğal Dil Araç Seti)

In [ ]:
# !pip install nltk

In [ ]:
# nltk'yi ilk kez içe aktarırken, birkaç yerleşik kütüphaneyi de indirmemiz gerekir.

import nltk

nltk.download('stopwords')
nltk.download('punkt')      # nltk<3.9.0 için
nltk.download('punkt_tab')  # nltk>=3.9.0 için
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
import pandas as pd

df = pd.read_csv("https://d32aokrjazspmn.cloudfront.net/materials/ham_spam_emails.csv")
df.head()

## (1) (Metin) veri setinin temizlenmesi

Veri kümesi, ham [0] veya spam [1] olarak sınıflandırılan e-postalardan oluşur. Tahmin modelini eğitmeden önce veri kümesini temizlemeniz gerekir.

### (1.1) Noktalama İşaretlerini Kaldır

❓ Noktalama işaretlerini kaldırmak için bir işlev oluşturun. Bunu `text` sütununa uygulayın ve çıktıyı `clean_text` adlı veri çerçevesinin yeni bir sütununa ekleyin. ❓

In [ ]:
# SENİN KODUN BURAYA

### (1.2) Küçük Harf

❓ Metni küçük harfe çeviren bir işlev oluşturun. Bunu `clean_text`'e uygulayın ❓

In [ ]:
# SENİN KODUN BURAYA

### (1.3) Sayıları Kaldır

❓ Metinden sayıları kaldırmak için bir işlev oluşturun. Bunu `clean_text`'e uygulayın ❓

In [ ]:
# SENİN KODUN BURAYA

### (1.4) StopWords'ü kaldırın

❓ Metinden durdurma kelimelerini kaldırmak için bir işlev oluşturun. Bunu `clean_text`'e uygulayın. ❓

In [ ]:
# SENİN KODUN BURAYA

### (1.5) Lemmatize

❓ Metni lemmatize etmek için bir fonksiyon oluşturun. Çıktının bir kelime listesi değil, tek bir dize olduğundan emin olun. Bunu `clean_text`'e uygulayın. ❓

In [ ]:
# SENİN KODUN BURAYA

## (2) Bag-of-Words Modellemesi

### (2.1) Metin verilerini sayılara dönüştürme

❓ `clean_text`'i varsayılan CountVectorizer ile Bag-of-Words temsiline vektörleştirin. `X_bow` olarak kaydedin. ❓

In [ ]:
# SENİN KODUN BURAYA

### (2.2) Çok terimli Naive Bayes Modellemesi

❓ MultinomialNB modelini bag-of-words verileriyle çapraz doğrulayın. Modelin doğruluğunu puanlayın. ❓

In [ ]:
# SENİN KODUN BURAYA

🏁 Tebrikler!

💾 Not defterinizi git add/commit/push yapmayı unutmayın...

🚀 ... ve bir sonraki challenge'a geçin!